Checking that missing dates are actually missing like we think

In [ ]:
from utils_filter import *
from utils_datetime import *
from utils_geography import *
from utils_plotting import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

Checking in grid outlooks

In [ ]:
missing_dates = ['200204190000', '200204200000', '200204210000', '200204250000', '200205060000', '200205250000', '200207310000', '200208130000', '200208300000', '200211090000', '200212230000', '200302030000', '200303250000', '200304140000', '200304150000', '200304160000', '200305100000', '200306250000', '200306280000', '200307270000', '200307280000', '200309030000', '200312280000', '200404020000', '200404140000', '200405230000', '200408090000', '200410140000', '200503300000', '200506060000', '200508030000', '200701040000', '200905280000', '201105210000', '202005240000', '202106130000']

In [ ]:
grid_outlook_location = 'data/outlooks/grid_outlooks.nc'
grid_outlooks = xr.open_dataset(grid_outlook_location)

In [ ]:
max_outlooks = grid_outlooks.max(dim = ['x', 'y'])['prob']

dates = max_outlooks['time']
thresholds = {'': .15,
              ' Tornado': .05,
              ' Wind': .15,
              ' Hail': .15}
days = ['Day 3', 'Day 2 7', 'Day 2 17', 'Day 1']
hazard_days = ['Day 2 7', 'Day 2 17', 'Day 1']
investigate_dates = []
for date in dates:
    max_outlooks_date = max_outlooks.sel(time = date)
    for hazard in ['', ' Tornado', ' Wind', ' Hail']:
        if hazard == '':
            this_days = days
        else:
            this_days = hazard_days
        outlook_keys = [item + hazard for item in this_days]
        max_prob = max_outlooks_date.sel(outlook = outlook_keys).max()
        if max_prob >= thresholds[hazard]:
            for outlook_key in outlook_keys:
                if max_outlooks_date.sel(outlook = outlook_key) == 0 and outlook_key == 'Day 1':
                    print(str(date.data) + ' ' + str(outlook_key))
                    investigate_dates.append(str(date.data))

                

print(investigate_dates)

In [ ]:
print(missing_dates)
print(investigate_dates)
len(investigate_dates)

In [ ]:
pph = xr.open_dataset('data/pph/labelled_pph.nc')
investigate_pph = pph.sel(time = investigate_dates)

In [ ]:
mdt_investigate_dates = investigate_pph['MAX_CAT'].isin(['MDT', 'HIGH'])
mdt_investigate_dates
mdt_investigate_pph = investigate_pph.sel(time = mdt_investigate_dates)
mdt_investigate_pph

Checking in raw data

In [ ]:
raw_dir = '~/Downloads'
outlook_raw_location = raw_dir + '/outlooks'

# Read convective outlooks into outlooks
year_list = [[2008, 2015]]
for years, i in zip(year_list, range(len(year_list))):
    print('reading file ' + str(i) + ', years ' + str(years[0]) +'-' + str(years[1]))
    if i == 0:
        outlooks_original = gp.read_file(outlook_raw_location + '/outlooks_' + str(years[0]) + '01010000_' + str(years[1]) + '12312359')
    else:
        outlooks_original = outlooks_original.append(gp.read_file(outlook_raw_location + '/outlooks_' + str(years[0]) + '01010000_' + str(years[1]) + '12312359'))
print('files read')
    
outlooks_original

In [ ]:
for date in investigate_dates:
    print(date)
    print(outlooks_original[(outlooks_original['ISSUE'] == date[0:8] + '1200') & (outlooks_original['DAY'] == 1)]['CYCLE'].unique())
